In [4]:
import numpy as np
from numpy import eye as eye
from numpy.linalg import inv as inv
from numpy.linalg import solve as solve
from numpy.linalg import eig as eig
from numpy.linalg import matrix_power

from numpy import random
from numpy.random import randn

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import scipy.optimize
 
import requests
import urllib

import time

In [5]:

# For this assignment, assume that every hidden layer has the same number of neurons.
NUM_INPUT = 784
NUM_OUTPUT = 10


In [6]:

def load_data():
    urllib.request.urlretrieve('https://s3.amazonaws.com/jrwprojects/fashion_mnist_train_images.npy', 'fashion_mnist_train_images.npy')
    urllib.request.urlretrieve('https://s3.amazonaws.com/jrwprojects/fashion_mnist_test_images.npy', 'fashion_mnist_test_images.npy')
    urllib.request.urlretrieve('https://s3.amazonaws.com/jrwprojects/fashion_mnist_train_labels.npy', 'fashion_mnist_train_labels.npy')
    urllib.request.urlretrieve('https://s3.amazonaws.com/jrwprojects/fashion_mnist_test_labels.npy', 'fashion_mnist_test_labels.npy')

    X_tr = np.reshape(np.load('fashion_mnist_train_images.npy'), (-1, 28*28))
    X_te = np.reshape(np.load('fashion_mnist_test_images.npy'), (-1, 28*28))
    y_tr = np.load('fashion_mnist_train_labels.npy')
    y_te = np.load('fashion_mnist_test_labels.npy')

    return ([X_tr, X_te, y_tr, y_te])



In [7]:

def one_hot_encode_y(y_tr_scalar, y_te_scalar, max_num):
    y_tr = (np.eye(max_num)[y_tr_scalar]).astype(int)
    y_te = (np.eye(max_num)[y_te_scalar]).astype(int)
    return ([y_tr, y_te])


In [8]:

def normalize_inputs(X, max_val):
    X = X/max_val
    return X


In [9]:

# def forward_prop_with_output (x, y, weightsAndBiases):
#     Ws, bs = unpack_new(weightsAndBiases)

#     x = np.atleast_2d(x)
#     y = np.atleast_2d(y)

#     print("x:", x.shape)
#     print("y:", y.shape)
#     print()

#     hs = []
#     zs = []

#     # set h0 equal to the input x.
#     hs.append(x.T)
#     # make dummy element for z[0] so indexing starts at 1
#     zs.append(x.T)  
#     # zs.append(np.zeros([hs[0].shape[0],hs[0].shape[1]]))

#     print("Input layer")
#     print("k:", 0)
#     print("zs[", 0, "]  ", zs[0].shape, sep="")
#     print("hs[", 0, "]  ", hs[0].shape, sep="")
#     print()

#     l = NUM_HIDDEN_LAYERS + 1

#     print("Hidden layers")    
#     for k in range(1, l):
#       b = np.atleast_2d(bs[k-1]).T
#       zs.append(Ws[k-1].dot(hs[k-1]) + b) 
#       hs.append(ReLU(zs[k]))
#       print("k:", k)
#       print("Ws[", k-1, "]  ", Ws[k-1].shape, sep="")
#       print("bs[", k-1, "]  ", bs[k-1].shape, sep="")
#       print("zs[", k, "]  ", zs[k].shape, sep="")
#       print("hs[", k, "]  ", hs[k].shape, sep="")
#       print()

#     # output layer
#     b = np.atleast_2d(bs[l-1]).T
#     zs.append(Ws[l-1].dot(hs[l-1]) + b)   
#     yhat = softmax(zs[l]) 

#     print("Output layer")
#     print("k:", k)
#     print("Ws[", l-1, "]  ", Ws[l-1].shape, sep="")
#     print("bs[", l-1, "]  ", bs[l-1].shape, sep="")
#     print("zs[", l, "]  ", zs[l].shape, sep="")
#     print("yhat", yhat.shape)
#     print()

#     loss = fCE(y, yhat)
#     print("fCE loss:", loss)

#     # Return loss, pre-activations, post-activations, and predictions
#     return loss, zs, hs, yhat
   

In [10]:

# def back_prop_with_output (x, y, weightsAndBiases):
#     # dimensions:             indexes:
#     # x: (m x n)
#     # y: (c x n)
#     # zs (outputs x n)        1 ... l
#     # hs (outputs x n)        1 ... l 
#     # Ws (outputs x inputs)   0 ... l-1
#     # bs (outputs)            0 ... l-1

#     loss, zs, hs, yhat = forward_prop(x, y, weightsAndBiases)
#     Ws, bs = unpack_new(weightsAndBiases)
    
#     x = np.atleast_2d(x)
#     y = np.atleast_2d(y)

#     y = y.T
#     g = yhat - y

#     n = y.shape[1]
#     l = NUM_HIDDEN_LAYERS + 1

#     dJdWs = []  # Gradients w.r.t. weights
#     dJdbs = []  # Gradients w.r.t. biases

#     print("NUM_HIDDEN_LAYERS:", NUM_HIDDEN_LAYERS)
#     print("l:", l)
#     print("y:", y.shape)
#     print("yhat:", yhat.shape)
#     print("g:", g.shape)
#     print()

#     print("--------------------------")
#     print()

#     for k in range(l, 0, -1):
#       print("k:", k)
#       try:
#         print("hs[", k, "]:  ", hs[k].shape, sep="")
#       except:
#         pass
#       try:
#         print("hs[", k-1, "]:  ", hs[k-1].shape, sep="")
#       except:
#         pass
#       try:
#         print("zs[", k, "]:  ", zs[k].shape, sep="")
#       except:
#         pass
#       try:
#         print("zs[", k-1, "]:  ", zs[k-1].shape, sep="")
#       except:
#         pass
#       try:
#         print("Ws[", k-1, "]:  ", Ws[k-1].shape, sep="")
#       except:
#         pass
#       try:
#         print("bs[", k-1, "]:  ", bs[k-1].shape, sep="")
#       except:
#         pass
#       print("g:", g.shape)

#       dJdWs.append(g.dot(hs[k-1].T)/n)  # + reg grad       
#       # dJdWs.append(hs[k-1].dot(g.T)/n)  # + reg grad       # old unpack 
#       dJdbs.append(np.mean(g, axis=1))

#       print("dJdWs[", k-1, "]:  ", dJdWs[len(dJdWs)-1].shape, sep="")
#       print("dJdbs[", k-1, "]:  ", dJdbs[len(dJdbs)-1].shape, sep="")
#       print()
      
#       if (k > 1):
#         g = Ws[k-1].T.dot(g)
#         # g = Ws[k-1].dot(g)       # old unpack 
#         g = g * ReLU_prime(zs[k-1])

#     print("--------------------------")
#     print()

#     # reverse order
#     dJdWs = dJdWs[::-1]
#     dJdbs = dJdbs[::-1]
#     print("dJdWs len:", len(dJdWs))
#     print("dJdbs len:", len(dJdbs))

#     # # Concatenate gradients
#     return np.hstack([ dJdW.flatten() for dJdW in dJdWs ] + [ dJdb.flatten() for dJdb in dJdbs ]) 


In [11]:

def initWeightsAndBiases_new(NUM_HIDDEN_LAYERS, NUM_HIDDEN):
    Ws = []
    bs = []

    np.random.seed(0)
    W = 2*(np.random.random(size=(NUM_HIDDEN, NUM_INPUT))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
    Ws.append(W)
    b = 0.01 * np.ones(NUM_HIDDEN)
    bs.append(b)

    for i in range(NUM_HIDDEN_LAYERS - 1):
        W = 2*(np.random.random(size=(NUM_HIDDEN, NUM_HIDDEN))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
        Ws.append(W)
        b = 0.01 * np.ones(NUM_HIDDEN)
        bs.append(b)

    W = 2*(np.random.random(size=(NUM_OUTPUT, NUM_HIDDEN))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
    Ws.append(W)
    b = 0.01 * np.ones(NUM_OUTPUT)
    bs.append(b)
    return np.hstack([ W.flatten() for W in Ws ] + [ b.flatten() for b in bs ])


In [12]:

def unpack_new (weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN):
    # Unpack arguments
    Ws = []

    # Weight matrices
    start = 0
    end = NUM_INPUT*NUM_HIDDEN
    W = weightsAndBiases[start:end]
    Ws.append(W)

    for i in range(NUM_HIDDEN_LAYERS - 1):
        start = end
        end = end + NUM_HIDDEN*NUM_HIDDEN
        W = weightsAndBiases[start:end]
        Ws.append(W)

    start = end
    end = end + NUM_HIDDEN*NUM_OUTPUT
    W = weightsAndBiases[start:end]
    Ws.append(W)

    Ws[0] = Ws[0].reshape(NUM_HIDDEN, NUM_INPUT)
    for i in range(1, NUM_HIDDEN_LAYERS):
        # Convert from vectors into matrices
        Ws[i] = Ws[i].reshape(NUM_HIDDEN, NUM_HIDDEN)
    Ws[-1] = Ws[-1].reshape(NUM_OUTPUT, NUM_HIDDEN)

    # Bias terms
    bs = []
    start = end
    end = end + NUM_HIDDEN
    b = weightsAndBiases[start:end]
    bs.append(b)

    for i in range(NUM_HIDDEN_LAYERS - 1):
        start = end
        end = end + NUM_HIDDEN
        b = weightsAndBiases[start:end]
        bs.append(b)

    start = end
    end = end + NUM_OUTPUT
    b = weightsAndBiases[start:end]
    bs.append(b)

    return Ws, bs


In [13]:

def training_validation_splits(X, Y, train_pct):
    split_num = int(np.round(X.shape[0] * train_pct,0))
    all_idx = random.permutation(X.shape[0])
    tr_idx, te_idx = all_idx[:split_num], all_idx[split_num:]
    X_tr, X_val = X[tr_idx,:], X[te_idx,:]
    Y_tr, Y_val = Y[tr_idx], Y[te_idx]
    return ([X_tr, X_val, Y_tr, Y_val])


In [14]:

def randomize_dataset_order(X, Y):
    all_idx = random.permutation(X.shape[0])
    return ([X[all_idx], Y[all_idx]])


In [15]:

def computeAccuracy (y, yhat):
    yhat = yhat.T
    return np.mean(np.argmax(y, axis=1) == np.argmax(yhat, axis=1))



In [16]:

def fCE (y, yhat, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA = 0.):
    Ws, bs = unpack_new(weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN)
    y = y.T
    L2_reg = ALPHA/y.shape[1] * np.sum([np.sum(W**2) for W in Ws])
    cost = - 1./y.shape[1] * np.sum(y * np.log(yhat)) + L2_reg
    return cost


In [17]:

def softmax(z):
    denom = np.sum(np.exp(z), axis=0, keepdims=True)
    return np.exp(z) / denom


In [18]:

def ReLU(z):
    return np.maximum(z,0)
    

In [19]:

def ReLU_prime(z):
    return np.where(z > 0, 1.0, 0.0)


In [20]:

def forward_prop (x, y, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA):

    Ws, bs = unpack_new(weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN)

    x = np.atleast_2d(x)
    y = np.atleast_2d(y)

    n = x.shape[0]
    hs = []
    zs = []

    # set h0 equal to the input x.
    hs.append(x.T)
    # make dummy element for z[0] so indexing starts at 1
    zs.append(x.T)  

    l = NUM_HIDDEN_LAYERS + 1
    for k in range(1, l):
      b = np.atleast_2d(bs[k-1]).T
      zs.append(Ws[k-1].dot(hs[k-1]) + b) 
      hs.append(ReLU(zs[k]))

    # output layer
    b = np.atleast_2d(bs[l-1]).T
    zs.append(Ws[l-1].dot(hs[l-1]) + b)   
    yhat = softmax(zs[l]) 

    loss = fCE(y, yhat, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)

    # Return loss, pre-activations, post-activations, and predictions
    return loss, zs, hs, yhat
   

In [21]:

def back_prop (x, y, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA):  
    # dimensions:             indexes:
    # x: (m x n)
    # y: (c x n)
    # zs (outputs x n)        1 ... l
    # hs (outputs x n)        1 ... l 
    # Ws (outputs x inputs)   0 ... l-1
    # bs (outputs)            0 ... l-1

    loss, zs, hs, yhat = forward_prop(x, y, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
    Ws, bs = unpack_new(weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN)

    x = np.atleast_2d(x)
    y = np.atleast_2d(y)
    y = y.T

    n = y.shape[1]
    l = NUM_HIDDEN_LAYERS + 1

    dJdWs = []  # Gradients w.r.t. weights
    dJdbs = []  # Gradients w.r.t. biases

    g = yhat - y

    for k in range(l, 0, -1):
      dJdWs.append(g.dot(hs[k-1].T)/n)  # + reg grad       
      dJdbs.append(np.mean(g, axis=1))  
      if (k > 1):
        g = Ws[k-1].T.dot(g)
        g = g * ReLU_prime(zs[k-1])

    # reverse order
    dJdWs = dJdWs[::-1]
    dJdbs = dJdbs[::-1]

    return (np.hstack([ dJdW.flatten() for dJdW in dJdWs ] + [ dJdb.flatten() for dJdb in dJdbs ]))


In [22]:

def plotSGDPath (trainX, trainY, trajectory):
    # TODO: change this toy plot to show a 2-d projection of the weight space
    # along with the associated loss (cross-entropy), plus a superimposed 
    # trajectory across the landscape that was traversed using SGD. Use
    # sklearn.decomposition.PCA's fit_transform and inverse_transform methods.

    def toyFunction (x1, x2):
        return np.sin((2 * x1**2 - x2) / 10.)

    fig = plt.figure()
    ax = fig.gca(projection='3d')

    # Compute the CE loss on a grid of points (corresonding to different w).
    axis1 = np.arange(-np.pi, +np.pi, 0.05)  # Just an example
    axis2 = np.arange(-np.pi, +np.pi, 0.05)  # Just an example
    Xaxis, Yaxis = np.meshgrid(axis1, axis2)
    Zaxis = np.zeros((len(axis1), len(axis2)))
    for i in range(len(axis1)):
        for j in range(len(axis2)):
            Zaxis[i,j] = toyFunction(Xaxis[i,j], Yaxis[i,j])
    ax.plot_surface(Xaxis, Yaxis, Zaxis, alpha=0.6)  # Keep alpha < 1 so we can see the scatter plot too.

    # Now superimpose a scatter plot showing the weights during SGD.
    Xaxis = 2*np.pi*np.random.random(8) - np.pi  # Just an example
    Yaxis = 2*np.pi*np.random.random(8) - np.pi  # Just an example
    Zaxis = toyFunction(Xaxis, Yaxis)
    ax.scatter(Xaxis, Yaxis, Zaxis, color='r')

    plt.show()


In [23]:

def train (trainX, trainY, testX, testY, size=60000,
           NUM_HIDDEN_LAYERS=3, NUM_HIDDEN=10, NUM_EPOCHS=1, BATCH_SIZE=1, ALPHA=0, LEARNING_RATE=1):

    # use subset of data for faster testing
    try: 
      trainX, trainY = randomize_dataset_order(trainX, trainY)
      trainX = trainX[0:size]
      trainY = trainY[0:size]
    except:
      pass

    # make train/val/test splits (80% train vs val)
    X_tr, X_val, Y_tr, Y_val  = training_validation_splits(trainX, trainY, .8)

    trajectory = []
    weightsAndBiases = initWeightsAndBiases_new(NUM_HIDDEN_LAYERS, NUM_HIDDEN) 
    l = NUM_HIDDEN_LAYERS + 1

    print("\nTraining -",
      "  NUM_HIDDEN_LAYERS:", NUM_HIDDEN_LAYERS, 
      "  NUM_HIDDEN:", NUM_HIDDEN,  
      "  NUM_EPOCHS:",NUM_EPOCHS, 
      "  BATCH_SIZE:", BATCH_SIZE,  
      "  ALPHA:", ALPHA, 
      "  LEARNING_RATE:", LEARNING_RATE, 
      "  size:", size)

    for epoch in range(NUM_EPOCHS):
      X_tr, Y_tr = randomize_dataset_order(X_tr, Y_tr)

      for i in range(0, X_tr.shape[0], BATCH_SIZE): 
        # print("Cummulative batch:", i)
        grads = back_prop(X_tr, Y_tr, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
        dJdWs, dJdbs = unpack_new(grads, NUM_HIDDEN_LAYERS, NUM_HIDDEN)
        Ws, bs = unpack_new(weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN)

        # update weights and biases
        for k in range(0, l):
          Ws[k] = Ws[k] - LEARNING_RATE * dJdWs[k] 
          bs[k] = bs[k] - LEARNING_RATE * dJdbs[k] 

        weightsAndBiases = np.hstack([ W.flatten() for W in Ws ] + [ b.flatten() for b in bs ])
        
        # collect training loss for trajectory
        loss_tr, zs, hs, yhat = forward_prop(X_val, Y_val, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
        trajectory.append([weightsAndBiases, loss_tr])
        acc = computeAccuracy(Y_val, yhat)
        
      # print("epoch:", epoch, " loss:", loss_tr, " acc:", acc)

    # validation loss and accuracy
    loss_val, zs, hs, yhat = forward_prop(X_val, Y_val, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
    acc = computeAccuracy(Y_val, yhat)

    print("validation loss:", loss_val, " acc:", acc)

    return loss_val, acc, trajectory


In [24]:

def test (X_tr, Y_tr, X_te, Y_te, 
          NUM_HIDDEN_LAYERS=3, NUM_HIDDEN=10, NUM_EPOCHS=1, BATCH_SIZE=1, ALPHA=0, LEARNING_RATE=1):

    weightsAndBiases = initWeightsAndBiases_new(NUM_HIDDEN_LAYERS, NUM_HIDDEN) 
    l = NUM_HIDDEN_LAYERS + 1

    for epoch in range(NUM_EPOCHS):
      X_tr, Y_tr = randomize_dataset_order(X_tr, Y_tr)

      for i in range(0, X_tr.shape[0], BATCH_SIZE): 
        # print("Cummulative batch:", i)
        grads = back_prop(X_tr, Y_tr, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
        dJdWs, dJdbs = unpack_new(grads, NUM_HIDDEN_LAYERS, NUM_HIDDEN)
        Ws, bs = unpack_new(weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN)

        # update weights and biases
        for k in range(0, l):
          Ws[k] = Ws[k] - LEARNING_RATE * dJdWs[k] 
          bs[k] = bs[k] - LEARNING_RATE * dJdbs[k] 

        weightsAndBiases = np.hstack([ W.flatten() for W in Ws ] + [ b.flatten() for b in bs ])
        

    # test loss and accuracy
    loss, zs, hs, yhat = forward_prop(X_te, Y_te, weightsAndBiases, NUM_HIDDEN_LAYERS, NUM_HIDDEN, ALPHA)
    acc = computeAccuracy(Y_te, yhat)

    print("epoch:", epoch, " loss:", loss, " acc:", acc)

    return loss, acc


In [25]:
def findBestHyperparameters(trainX, trainY, testX, testY,
                            size=60000,  # number of data samples to include
                            num_hidden_layers_list = [3],
                            num_hidden_list = [10],
                            num_epochs_list = [1],
                            batch_size_list = [256],
                            alpha_list = [0],
                            learning_rate_list = [.001]):

    tuning_log = []
    current_iter = 0
    max_iters = (len(num_hidden_layers_list) *
                 len(num_hidden_list) *
                 len(num_epochs_list) *
                 len(batch_size_list) *
                 len(alpha_list) *
                 len(learning_rate_list))

    for NUM_HIDDEN_LAYERS in num_hidden_layers_list:
      for NUM_HIDDEN in num_hidden_list:
        for NUM_EPOCHS in num_epochs_list:
          for BATCH_SIZE in batch_size_list:
            for ALPHA in alpha_list:
              for LEARNING_RATE in learning_rate_list:

                t0 = time.time()

                loss, acc,  trajectory = train(trainX, trainY, testX, testY, size,
                                               NUM_HIDDEN_LAYERS, NUM_HIDDEN, 
                                               NUM_EPOCHS, BATCH_SIZE, ALPHA, LEARNING_RATE)
                
                t1 = time.time()

                print("Loss:", loss,
                      "  acc:", acc, 
                      "  NUM_HIDDEN_LAYERS:", NUM_HIDDEN_LAYERS, 
                      "  NUM_HIDDEN:", NUM_HIDDEN,  
                      "  NUM_EPOCHS:",NUM_EPOCHS, 
                      "  BATCH_SIZE:", BATCH_SIZE,  
                      "  ALPHA:", ALPHA, 
                      "  LEARNING_RATE:", LEARNING_RATE, 
                      "  elapsed:", t1-t0)

                # if the current loss results are the best so far, record these hyperparameters
                if (current_iter>0):  
                  if (loss < best_params['loss']):
                    best_params = {'NUM_HIDDEN_LAYERS': NUM_HIDDEN_LAYERS,
                                  'NUM_HIDDEN': NUM_HIDDEN,
                                  'NUM_EPOCHS': NUM_EPOCHS,
                                  'BATCH_SIZE': BATCH_SIZE,
                                  'ALPHA': ALPHA,
                                  'LEARNING_RATE': LEARNING_RATE,
                                  'loss': loss,
                                  'accuracy': acc} 
                else:
                  best_params =  {'NUM_HIDDEN_LAYERS': NUM_HIDDEN_LAYERS,
                                  'NUM_HIDDEN': NUM_HIDDEN,
                                  'NUM_EPOCHS': NUM_EPOCHS,
                                  'BATCH_SIZE': BATCH_SIZE,
                                  'ALPHA': ALPHA,
                                  'LEARNING_RATE': LEARNING_RATE,
                                  'loss': loss,
                                  'accuracy': acc} 

                # log tuning results for after analysis
                tuning_results = {'NUM_HIDDEN_LAYERS': NUM_HIDDEN_LAYERS,
                                  'NUM_HIDDEN': NUM_HIDDEN,
                                  'NUM_EPOCHS': NUM_EPOCHS,
                                  'BATCH_SIZE': BATCH_SIZE,
                                  'ALPHA': ALPHA,
                                  'LEARNING_RATE': LEARNING_RATE,
                                  'loss': loss,
                                  'accuracy': acc}

                tuning_log.append(tuning_results)
                current_iter = current_iter + 1


    #completed testing all hyperparameters - output summary results
    print('\n Hyperparameters tested:', 
          '\n   num_hidden_layers_list:', num_hidden_layers_list,
          '\n   num_hidden_list:', num_hidden_list,
          '\n   num_epochs_list:', num_epochs_list,
          '\n   batch_size_list:', batch_size_list,
          '\n   alpha_list:', alpha_list,
          '\n   learning_rate:', learning_rate_list)

    print('\n best_params:', 
          '\n   NUM_HIDDEN_LAYERS:', best_params['NUM_HIDDEN_LAYERS'],
          '\n   NUM_HIDDEN:', best_params['NUM_HIDDEN'],
          '\n   NUM_EPOCHS:', best_params['NUM_EPOCHS'],
          '\n   BATCH_SIZE:', best_params['BATCH_SIZE'],
          '\n   ALPHA:', best_params['ALPHA'],
          '\n   learning_rate:', best_params['LEARNING_RATE'],
          '\n   loss:', best_params['loss'],
          '\n   accuracy:', best_params['accuracy'])

    return (tuning_log, best_params)


In [26]:

if __name__ == "__main__":
    # load data
    trainX, testX, trainY, testY = load_data()

    # Y class one-hot encoding
    trainY, testY = one_hot_encode_y(trainY, testY, 10)

    # Normalize X pixel values
    trainX = normalize_inputs(trainX, 255)
    testX = normalize_inputs(testX, 255)

    # Initialize weights and biases randomly
    weightsAndBiases = initWeightsAndBiases_new(NUM_HIDDEN_LAYERS=3, NUM_HIDDEN=10)
    
    
    # # # Perform gradient check on random training examples
    # print(scipy.optimize.check_grad(lambda wab: forward_prop(np.atleast_2d(trainX[0:5,:]), np.atleast_2d(trainY[0:5,:]), wab)[0], \
    #                                 lambda wab: back_prop(np.atleast_2d(trainX[0:5,:]), np.atleast_2d(trainY[0:5,:]), wab), \
    #                                 weightsAndBiases))


    print(scipy.optimize.check_grad(lambda wab: forward_prop(np.atleast_2d(trainX[0:5,:]), 
                                                            np.atleast_2d(trainY[0:5,:]), 
                                                            wab,
                                                            NUM_HIDDEN_LAYERS=3, NUM_HIDDEN=10, ALPHA=0)[0],
                                    lambda wab: back_prop(np.atleast_2d(trainX[0:5,:]), 
                                                          np.atleast_2d(trainY[0:5,:]), 
                                                          wab,
                                                          NUM_HIDDEN_LAYERS=3, NUM_HIDDEN=10, ALPHA=0), 
                                    weightsAndBiases))


    
    # Plot the SGD trajectory
    # plotSGDPath(trainX, trainY, ws)
    pass

8.410416608067793e-07


In [27]:

# results, best_params = findBestHyperparameters(trainX, trainY, testX, testY, size=2500,
#                                               num_hidden_layers_list = [3,4,5],
#                                               num_hidden_list = [10, 30],
#                                               num_epochs_list = [10, 20],
#                                               batch_size_list = [16, 64, 256],
#                                               alpha_list = [0.01, .05, .1],
#                                               learning_rate_list = [.001, .01, .1])    


In [ ]:

findBestHyperparameters(trainX, trainY, testX, testY, size=60000,
                        num_hidden_layers_list = [4],
                        num_hidden_list = [30],
                        num_epochs_list = [10],
                        batch_size_list = [16],
                        alpha_list = [.01],
                        learning_rate_list = [.1])    



Training -   NUM_HIDDEN_LAYERS: 4   NUM_HIDDEN: 30   NUM_EPOCHS: 10   BATCH_SIZE: 16   ALPHA: 0.01   LEARNING_RATE: 0.1   size: 60000
